In [3]:
!pip install numpy-financial
import pandas as pd
import numpy as np
import numpy_financial as npf
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
from google.colab import drive

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd

# Ruta del archivo en Drive
ruta_excel = '/content/drive/MyDrive/Colab Notebooks/Flujos_de_caja.xlsx'

# Leer Excel (usualmente la primera hoja)
df = pd.read_excel(ruta_excel)

# Verificar el contenido
df.head()

,Proyecto,Inversion_Inicial,Año1,Año2,Año3,Año4,Año5,Año6,Año7,Año8,Año9,Año10
0,Winston Smith,-85682867,5570451,16720657,7801445,18051670,15958457,12517201,5862839,20146580,19444292,24566746
1,Julia,-76755036,5967344,6490687,17913082,5955205,17184923,24685303,20474106,17186651,9233384,13512686
2,O'Brien,-76882282,16243667,18070601,16252748,15511699,20448040,5658139,15070853,13582851,17689154,18385842
3,Big Brother,-41081788,7412149,14311768,16953070,5966649,8775702,24747640,22841583,19143933,18412066,12835958
4,Don Quijote,-33315092,10398869,16991292,11839789,20617892,9090284,20860467,22012114,17840143,20194144,10923808


In [6]:
#tamaño de datos
print('\n El tamaño por filas y columnas',df.shape)


 El tamaño por filas y columnas (1000, 12)


In [7]:
df.dtypes
print('\n Los datos son de tipo:\n',df.dtypes)


 Los datos son de tipo:
 Proyecto             object
Inversion_Inicial     int64
Año1                  int64
Año2                  int64
Año3                  int64
Año4                  int64
Año5                  int64
Año6                  int64
Año7                  int64
Año8                  int64
Año9                  int64
Año10                 int64
dtype: object


In [8]:
df.head(5)

,Proyecto,Inversion_Inicial,Año1,Año2,Año3,Año4,Año5,Año6,Año7,Año8,Año9,Año10
0,Winston Smith,-85682867,5570451,16720657,7801445,18051670,15958457,12517201,5862839,20146580,19444292,24566746
1,Julia,-76755036,5967344,6490687,17913082,5955205,17184923,24685303,20474106,17186651,9233384,13512686
2,O'Brien,-76882282,16243667,18070601,16252748,15511699,20448040,5658139,15070853,13582851,17689154,18385842
3,Big Brother,-41081788,7412149,14311768,16953070,5966649,8775702,24747640,22841583,19143933,18412066,12835958
4,Don Quijote,-33315092,10398869,16991292,11839789,20617892,9090284,20860467,22012114,17840143,20194144,10923808


In [9]:
# Limpiar datos eliminando filas con valores nulos
df_clean = df.dropna()

# Asegurar que los valores numéricos estén correctamente interpretados
for col in df_clean.columns[1:]:
    df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

In [10]:
# Funciones financieras: VPN y TIR
def calcular_vpn(flujos, tasa_descuento):
    return np.sum([fc / (1 + tasa_descuento) ** t for t, fc in enumerate(flujos)])

def calcular_tir(flujos):
    return npf.irr(flujos)

In [11]:
# Visualización de Resultados
def graficar_vpn_tir(resultados):
    proyectos = list(resultados.keys())
    vpns = [val['VPN'] for val in resultados.values()]
    tirs = [val['TIR'] * 100 for val in resultados.values()]

    fig, ax1 = plt.subplots(figsize=(12, 6))
    ax1.bar(proyectos, vpns, color='skyblue', label='VPN')
    ax1.set_ylabel('VPN ($)', color='skyblue')
    ax1.tick_params(axis='y', labelcolor='skyblue')
    ax1.set_xticks(range(len(proyectos)))
    ax1.set_xticklabels(proyectos, rotation=90)

    ax2 = ax1.twinx()
    ax2.plot(proyectos, tirs, color='red', marker='o', label='TIR')
    ax2.set_ylabel('TIR (%)', color='red')
    ax2.tick_params(axis='y', labelcolor='red')

    fig.tight_layout()
    plt.title('VPN y TIR por Proyecto')
    plt.grid(True)
    plt.show()

In [12]:
def resumen_resultados(resultados):
    for proyecto, valores in resultados.items():
        vpn = valores['VPN']
        tir = valores['TIR']
        print(f"{proyecto}: VPN = ${vpn:.2f}, TIR = {tir:.2%}")
        if vpn > 0:
            print(" -> Proyecto rentable ")
        else:
            print(" -> Proyecto NO rentable ")

In [13]:
# Menú Interactivo

import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt

def ejecutar_menu(df):
    tasa_input = widgets.FloatSlider(description='Tasa (%)', value=0.1, min=0.01, max=0.3, step=0.01)
    btn = widgets.Button(description='Calcular VPN/TIR')
    out = widgets.Output()

    def on_btn_click(b):
        with out:
            clear_output()
            tasa = tasa_input.value
            resultados = {}

            for i, row in df.iterrows():
                try:
                    flujos = [-row['Inversion_Inicial']] + [row[f'Año{j}'] for j in range(1, 11)]
                    resultados[row['Proyecto']] = {
                        'VPN': calcular_vpn(flujos, tasa),
                        'TIR': calcular_tir(flujos)
                    }
                except Exception as e:
                    print(f"Error en proyecto {row['Proyecto']}: {e}")

            # Mostrar resumen numérico
            resumen_resultados(resultados)

            # Mostrar gráfico
            graficar_vpn_tir(resultados)


    btn.on_click(on_btn_click)
    display(tasa_input, btn, out)


In [16]:
# Ejecutar el menú interactivo con los datos limpios
ejecutar_menu(df_clean)

FloatSlider(value=0.1, description='Tasa (%)', max=0.3, min=0.01, step=0.01)

Button(description='Calcular VPN/TIR', style=ButtonStyle())

Output()